## Computer Vision Base Trainer

The basic steps we'll take are:

1. Importing our collected, organized, and cleaned images
1. Fine-tune a pretrained neural network to recognise these two groups
1. Try running this model on a picture from our test dataset and see if it works.

## STEP 1: Ensure you are running this notebook within a Docker Container

View the README.md file for instructions on building and running a container.

In [ ]:
import zipfile
import os

# Define the path to the zip file and the extraction directory
zip_file_path = '/workspace/data/TRAINING_DATA.zip'
extract_to_dir = '/workspace/data/TRAINING_DATA'

# Check if the TRAINING_DATA directory already exists
if not os.path.exists(extract_to_dir):
    # Create the extraction directory if it doesn't exist
    os.makedirs(extract_to_dir, exist_ok=True)

    # Open the zip file and extract its contents
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)

    print(f"Extracted all files to {extract_to_dir}")
else:
    print(f"Directory {extract_to_dir} already exists. Skipping extraction.")


In [ ]:
from fastai.vision.all import *

## Step 2: Train our model

In [ ]:
path = '/workspace/data/TRAINING_DATA'

In [ ]:
# only run this test if you've added new data to the training data
# it does NOT need to be run if you're confident that the image files are valid 
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

To train a model, we'll need `DataLoaders`, which is an object that contains a *training set* (the images used to create a model) and a *validation set* (the images used to check the accuracy of a model -- not used during training). In `fastai` we can create that easily using a `DataBlock`, and view sample images from it:

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')],
    batch_tfms=aug_transforms(size=224)
).dataloaders(path, bs=256)

dls.show_batch(max_n=18)

Here what each of the `DataBlock` parameters means:

    blocks=(ImageBlock, CategoryBlock),

The inputs to our model are images, and the outputs are categories (in this case, "bird" or "forest").

    get_items=get_image_files, 

To find all the inputs to our model, run the `get_image_files` function (which returns a list of all image files in a path).

    splitter=RandomSplitter(valid_pct=0.2, seed=42),

Split the data into training and validation sets randomly, using 20% of the data for the validation set.

    get_y=parent_label,

The labels (`y` values) is the name of the `parent` of each file (i.e. the name of the folder they're in, which will be the names of each plant).

    item_tfms=[Resize(192, method='squish')]

Before training, resize each image to 192x192 pixels by "squishing" it (as opposed to cropping it).

    aug_transforms(size=224)

Data augmentation transformations, in this case, changing the output size to 224.

Now we're ready to train our model. The fastest widely used computer vision model is `resnet18`. You can train this in a few minutes, even on a CPU! (On a GPU, it generally takes under 10 seconds...)

`fastai` comes with a helpful `fine_tune()` method which automatically uses best practices for fine tuning a pre-trained model, so we'll use that.

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)

In [ ]:
learn.lr_find()

"Fine-tuning" a model means that we're starting with a model someone else has trained using some other dataset (called the *pretrained model*), and adjusting the weights a little bit so that the model learns to recognise your particular dataset. In this case, the pretrained model was trained to recognise photos in *imagenet*, and widely-used computer vision dataset with images covering 1000 categories) For details on fine-tuning and why it's important, check out the [free fast.ai course](https://course.fast.ai/).

In [ ]:
epochs = 3
learn.fine_tune(epochs, base_lr=0.0014454397605732083) #use the value from the lr_find() method above

In [ ]:
learn.show_results()

## Step 3: Let's see what data our model has the most difficulty with

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

If you are satisfied with your trained `Learner`, then you can export the file that contains all the information needed to run the model:

In [ ]:
learn.export('../models/model.pkl')